## aquí realizaremos el proceso de ETL al archivo

Importamos las librerías necesarias

In [1]:
import json
import pandas as pd
import ast
import numpy as np

In [29]:
# Inicializamo una lista vacía para almacenar las filas del archivo JSON
row = []

with open('australian_user_reviews.json', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        row.append(ast.literal_eval(line)) #evalúa expresiones literales de Python de una cadena de texto de manera segura

# Cargamos los datos en un DataFrame utilizando la lista de filas
df_reviews = pd.DataFrame(row)

#Mostramos las primeras 8 líneas del Dataframe
df_reviews.head(8)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
5,Wackky,http://steamcommunity.com/id/Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'..."
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,"[{'funny': '1 person found this review funny',..."
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,"[{'funny': '', 'posted': 'Posted July 24.', 'l..."


Procederemos a desanidar el archivo

In [30]:
#Este paso utiliza el método explode() para desanidar la columna 'reviews', la cual contiene listas o estructuras anidadas.
df_reviews_desanidado = df_reviews['reviews'].explode()

#Se reinicia el índice en los DataFrames originales (df_reviews y df_reviews_desanidado) 
# para que tengan un índice común que se pueda utilizar para fusionar los DataFrames.
df_reviews_reset = df_reviews.reset_index()
df_reviews_desanidado_reset = df_reviews_desanidado.reset_index()

#En este paso, los DataFrames reiniciados se fusionan utilizando el método merge(). Se utiliza el 
# índice como clave de fusión (left_on y right_on) para combinar los datos.
df_merge = df_reviews_reset.merge(df_reviews_desanidado_reset, left_on='index', right_on='index')

In [31]:
#mostramos las primeras 5 líneas del dataset
df_merge.head(5)

,index,user_id,user_url,reviews_x,reviews_y
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","{'funny': '', 'posted': 'Posted November 5, 20..."
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","{'funny': '', 'posted': 'Posted July 15, 2011...."
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","{'funny': '', 'posted': 'Posted April 21, 2011..."
3,1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","{'funny': '', 'posted': 'Posted June 24, 2014...."
4,1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","{'funny': '', 'posted': 'Posted September 8, 2..."


Eliminimos las columnas indeseadas generadas en el merge

In [32]:
# Eliminar las columnas index, user_url y reviews_x
df_merge = df_merge.drop(['index', 'user_url', 'reviews_x'], axis=1)

# Cambiar el nombre de la columna reviews_x a reviews
df_merge = df_merge.rename(columns={'reviews_y': 'reviews'})

Desanidaremos la columna reviews

In [33]:
# Convierte la columna 'reviews' en un DataFrame anidado
reviews_normalized = pd.json_normalize(df_merge['reviews'])

# Combina el nuevo DataFrame con el original
df_user_reviews = pd.concat([df_merge, reviews_normalized], axis=1)

# Elimina la columna original 'reviews'
df_user_reviews = df_user_reviews.drop('reviews', axis=1)

# Muestra el DataFrame resultante
df_user_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


Se nos pide utilizar un algoritmo NPL para analizar sentimientos en la columna reviews, en este caso utilizaremos NLTK (Natural Language Toolkit).

In [34]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')  #es una instrucción en NLTK (Natural Language Toolkit) que descarga 
#el lexicon de VADER (Valence Aware Dictionary and sEntiment Reasoner). VADER es una herramienta de 
# análisis de sentimientos diseñada para procesar texto en inglés y determinar la polaridad del 
# sentimiento, es decir, si el texto tiene una connotación positiva, negativa o neutral.

# Crear un objeto SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Función para realizar el análisis de sentimiento
def analyze_sentiment(review):
    # Obtener el puntaje de polaridad del sentimiento
    compound_score = sia.polarity_scores(str(review))['compound']

    # Clasificar la polaridad del sentimiento en 3 categorías: negativo, neutral, positivo
    if compound_score < 0:
        return 0  # Sentimiento negativo
    elif compound_score == 0:
        return 1  # Sentimiento neutral
    else:
        return 2  # Sentimiento positivo

# Aplicar el análisis de sentimiento a la columna 'review'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analyze_sentiment)

# Mostrar las primeras filas del DataFrame con la nueva columna
print(df_user_reviews.head())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


             user_id funny                     posted last_edited item_id  \
0  76561197970982479         Posted November 5, 2011.                1250   
1  76561197970982479            Posted July 15, 2011.               22200   
2  76561197970982479           Posted April 21, 2011.               43110   
3            js41637            Posted June 24, 2014.              251610   
4            js41637        Posted September 8, 2013.              227300   

                                           helpful recommend  \
0                                   No ratings yet      True   
1                                   No ratings yet      True   
2                                   No ratings yet      True   
3  15 of 20 people (75%) found this review helpful      True   
4     0 of 1 people (0%) found this review helpful      True   

                                              review  sentiment_analysis  
0  Simple yet with great replayability. In my opi...                   2  
1 

In [35]:
# Mostramos información sobre el dataset
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59333 non-null  object
 1   funny               59305 non-null  object
 2   posted              59305 non-null  object
 3   last_edited         59305 non-null  object
 4   item_id             59305 non-null  object
 5   helpful             59305 non-null  object
 6   recommend           59305 non-null  object
 7   review              59305 non-null  object
 8   sentiment_analysis  59333 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 4.1+ MB


Se pudo detectar que el Dataframe tiene muchos valores en blanco, para poder tener una idea de los valores nulos porcederemos a sustituir valores vacios por NaN y de esta manera poder contabilizarlos

In [36]:
# Reemplazar espacios en blanco por NaN en todo el DataFrame
df_user_reviews.replace('', np.nan, inplace=True)

# Imprimir información del DataFrame después de la operación
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59333 non-null  object
 1   funny               8151 non-null   object
 2   posted              59305 non-null  object
 3   last_edited         6140 non-null   object
 4   item_id             59305 non-null  object
 5   helpful             59305 non-null  object
 6   recommend           59305 non-null  object
 7   review              59275 non-null  object
 8   sentiment_analysis  59333 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 4.1+ MB


Eliminaremos las columnas funny, last_edited, helpful, review, solo conservaremos de las columnas generadas a partir de reviews item_id y recomend, las demás generan espacio en memoria y no tienen ningún aporte.

In [37]:
df_user_reviews = df_user_reviews.drop(['funny', 'last_edited', 'helpful', 'review'], axis=1)
df_user_reviews

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,2
1,76561197970982479,"Posted July 15, 2011.",22200,True,2
2,76561197970982479,"Posted April 21, 2011.",43110,True,2
3,js41637,"Posted June 24, 2014.",251610,True,2
4,js41637,"Posted September 8, 2013.",227300,True,2
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,2
59329,76561198312638244,Posted July 8.,362890,True,2
59330,LydiaMorley,Posted July 3.,273110,True,2
59331,LydiaMorley,Posted July 20.,730,True,2


In [38]:
#mostramos información del dataframe
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59333 non-null  object
 1   posted              59305 non-null  object
 2   item_id             59305 non-null  object
 3   recommend           59305 non-null  object
 4   sentiment_analysis  59333 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.3+ MB


Debido a la poca cantidad de nulos procederemos a eliminarlos

In [39]:
df_user_reviews = df_user_reviews.dropna()
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   posted              59305 non-null  object
 2   item_id             59305 non-null  object
 3   recommend           59305 non-null  object
 4   sentiment_analysis  59305 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


Nuestro cliente nos encargo un sistema de recomendacion para el cual necesitamos el año que se encuenta en la columna posted, extraeremos el año de dicha columna y lo almacenaremos en una nueva columna llamada year, los campos que tengan la fecha incompleta como por ejemplo: Posted May 31. es decir, que no tengan el año se les asignara el valor 0, mas adelante aplicaremos una estrategia para esos valores,

In [ ]:
#  Utiliza expresiones regulares para extraer un patrón de cuatro dígitos que representan un año. 
# El argumento expand=False devuelve la salida como una Serie en lugar de un DataFrame.
df_user_reviews['year'] = df_user_reviews['posted'].str.extract(r'(\d{4})', expand=False)

# Aplica una función lambda a cada fila del DataFrame. La función verifica si el valor en la columna 
# 'year' es nulo; si es así, lo reemplaza con 0; de lo contrario, convierte el valor a tipo entero.
df_user_reviews['year'] = df_user_reviews.apply(lambda row: 0 if pd.isnull(row['year']) else int(row['year']), axis=1)


Ya con el año en la columna year, no necesitamos la columna posted

In [43]:
# Eliminar la columna original 'posted'
df_user_reviews = df_user_reviews.drop('posted', axis=1)

así quedo nuestro Dataframe

In [49]:
df_user_reviews

,user_id,item_id,recommend,sentiment_analysis,year
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,2,2011
2,76561197970982479,43110,True,2,2011
3,js41637,251610,True,2,2014
4,js41637,227300,True,2,2013
...,...,...,...,...,...
59328,76561198312638244,70,True,2,0
59329,76561198312638244,362890,True,2,0
59330,LydiaMorley,273110,True,2,0
59331,LydiaMorley,730,True,2,0


In [50]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   item_id             59305 non-null  object
 2   recommend           59305 non-null  object
 3   sentiment_analysis  59305 non-null  int64 
 4   year                59305 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.7+ MB


Exportamos el Dataframe a un archivo csv

In [51]:
df_user_reviews.to_csv('./user_reviews.csv', index=False)

Por ahora hemos concluido con el ETL del dataset 'australian_user_reviews.json', profundizaremos mas en estos datos las realizar el proceso de EDA.